# Filtragem baseada em conteúdo: TF-IDF e LDA

In [1]:
# Importação 
import pickle
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize

# Carregando o dataset
vinhos_dados_arq_csv = 'data/XWines_Slim_1K_wines.csv'
vinhos_avals_arq_csv = 'data/XWines_Slim_150K_ratings.csv'

vinhos_dados = pd.read_csv(vinhos_dados_arq_csv)
vinhos_avals = pd.read_csv(vinhos_avals_arq_csv, low_memory=False)

# Organização dos dados

- merge dos arquivos de dataset
- limpar atributos 
- criar um atributo "description"
- extrair a descrição

In [2]:
# Merge dos dataset
vinhos = pd.merge(vinhos_dados, vinhos_avals, on='WineID')

# Limpar os atributos Grapes e Harmonize 
uvas = vinhos['Grapes'].str.split(',', expand=True).stack().str.replace('[', '').str.replace(']', '').str.strip()
uvas = uvas.reset_index(level=1, drop=True).rename('Grape')

harm = vinhos['Harmonize'].str.split(',', expand=True).stack().str.replace('[', '').str.replace(']', '').str.strip()
harm = harm.reset_index(level=1, drop=True).rename('Harmonize1')


vinhos = vinhos.join(uvas)
vinhos = vinhos.join(harm)


# Criar o atributo description

vinhos['description'] = vinhos['WineName'] + ' ' + vinhos['Type'] + ' ' + vinhos['Grape'] + ' ' + vinhos['Country'] + ' ' + vinhos['RegionName'] + ' ' + vinhos['WineryName'] + ' Harmoniza com  ' + vinhos['Harmonize1']

# Extrair as descrições

descriptions = vinhos['description']


# TF-IDF

In [3]:
# Criar o vetorizador TF-IDF com suporte a stopwords em inglês e português
vectorizer_tfidf = TfidfVectorizer(stop_words=['english', 'portuguese'])
tfidf_matrix = vectorizer_tfidf.fit_transform(descriptions)

# LDA

In [4]:
vectorizer_lda = CountVectorizer(stop_words=['english', 'portuguese'])
word_count = vectorizer_lda.fit_transform(descriptions)

In [5]:
# Criar o modelo LDA
lda = LatentDirichletAllocation(n_components=10, random_state=0)
# lda.fit(word_count)

In [8]:
# Salvar o modelo
# pickle.dump(lda, open('models/lda_model.pkl', 'wb'))

# Carregar o modelo
lda = pickle.load(open('models/lda_model.pkl', 'rb'))

In [9]:
# Normalizar as matrizes 
tfidf_norm = normalize(tfidf_matrix)
lda_norm = normalize(lda.transform(word_count))

# Verificação das dimensões das matrizes
print("Dimensões de tfidf_norm: ", tfidf_norm.shape)
print("Dimensões de lda_norm: ", lda_norm.shape)

Dimensões de tfidf_norm:  (1010887, 2391)
Dimensões de lda_norm:  (1010887, 10)


# Perfil de conteúdo

In [10]:
# Converter a matriz esparsa tfidf_norm para um array Numpy denso
tfidf_norm_dense = tfidf_norm.toarray()

# Concatenar as matrizes tfidf_norm_dense e lda_norm
combined_features = np.concatenate((tfidf_norm_dense, lda_norm), axis=1)

# Verificação das dimensões da matriz combinada
print("Dimensões de combined_features: ", combined_features.shape)


Dimensões de combined_features:  (1010887, 2401)


# Perfil do usuário

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
# Construir o perfil do usuário 

usuario_id = 1000036

# Filtrar as avaliações do usuário
avaliacoes_usuario = vinhos[vinhos['UserID'] == usuario_id]

# Filtrar os vinhos avaliados pelo usuário que possuem boas avaliações
avaliacoes_positivas = avaliacoes_usuario[avaliacoes_usuario['Rating'] >= 4]

# Extrair as descrições dos vinhos avaliados positivamente
descricoes_positivas = avaliacoes_positivas['description']

# Usar o tf-idf e lda para transformar as descrições em vetores
tfidf_desc_positivas = vectorizer_tfidf.transform(descricoes_positivas)
lda_desc_positivas = lda.transform(vectorizer_lda.transform(descricoes_positivas))

# Normalizar os vetores
tfidf_desc_positivas_norm = normalize(tfidf_desc_positivas)
lda_desc_positivas_norm = normalize(lda_desc_positivas)

# Perfil do usuário 
perfil_usuario = np.mean(np.concatenate((tfidf_desc_positivas_norm.toarray(), 
                                         lda_desc_positivas_norm), axis=1), axis=0)

# Recomendação de vinhos

# Calcular a similaridade cosseno entre o perfil do usuário e os vinhos
similaridade = cosine_similarity([perfil_usuario], combined_features)

# Extrair os índices dos 10 vinhos mais similares
n_recomedacoes = 10
indices_recomendacoes = np.argsort(similaridade[0])[::-1][:n_recomedacoes]

recomendacoes = vinhos.iloc[indices_recomendacoes]

recomendacoes = recomendacoes.drop_duplicates(subset=['WineID'])

display(recomendacoes[['WineID', 'WineName', 'WineryName', 'Type', 'Country']])

,WineID,WineName,WineryName,Type,Country
117588,170984,Kadette Cape Blend,Kanonkop,Red,South Africa
